In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

import torch
from dqn_arch.model import *
import torchvision
from torchvision import datasets, transforms
import numpy as np
import torch.nn as nn

from itertools import product

device = 'cuda'

# CIFAR10

In [ ]:
fixed_transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

randomed_transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

fixed_trainset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=True, download=True, transform=fixed_transform_train)
fixed_trainloader = torch.utils.data.DataLoader(fixed_trainset, batch_size=100, shuffle=False, num_workers=2)

trainset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='~/data/cifar10', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)


In [ ]:
def test(net, loader):
    net.eval()
    correct = 0
    total =  0
    test_loss = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = loss_function(outputs,targets)
            test_loss.append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            
    acc = 100.*correct/total
    test_loss_mean = np.mean(test_loss)
    test_loss_std = np.std(test_loss)
    return test_loss_mean, test_loss_std, acc

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
def eval_plot_2d_landscape(net, nettype, loader, basis, r=1.0, scale=11,title='Title'):
    b1,b2 = basis
    x = np.linspace(-r, r, scale)
    y = np.linspace(-r, r,scale)
    nx,ny = len(x), len(y)
    loss_map = np.zeros((nx,ny))
    acc_map = np.zeros((nx,ny))
    std_map = np.zeros((nx,ny))

    tnet = nettype
    tnet.load_state_dict(net.state_dict())
    origin = tnet.get_weight_vector()    

    for i,j in product(range(nx),range(ny)):
        vec = origin + b1 * x[i] + b2 * y[j]
        tnet.set_weight_vector(vec)
        loss, std, acc = test(tnet, loader)
        loss_map[i, j] = loss
        acc_map[i, j] = acc
        std_map[i, j] = std
        print('\r',i, j, x[i], y[j], acc, loss,end='')
    print()
    X,Y = np.meshgrid(x,y)

    
    fig, axes = plt.subplots(1,2, figsize=(15,10))

    cs = axes[0].contour(X,Y,loss_map, levels=np.logspace(np.log(loss_map.min()), np.log(loss_map.max()),15,base=np.e))
    plt.clabel(cs, inline=1, fontsize=10)
    axes[0].set_title('Loss')
    cs = axes[1].contour(X,Y,std_map, levels=np.logspace(np.log(std_map.min()), np.log(std_map.max()),15,base=np.e))
    plt.clabel(cs, inline=1, fontsize=10)
    axes[1].set_title('Std')
    fig.suptitle(title)

    return loss_map, acc_map, std_map

In [ ]:
net = Model(in_size=(3,32,32))
checkpoint = torch.load('./dqn_arch/checkpoint/data_CIFAR10_optim_sgd_lr_0.01_wd_0.0005_batch-size_128_seed_2018/best.t7')
net.load_state_dict(checkpoint['net'])    
net.to(device)

In [ ]:
zero_vector = np.zeros(net.get_weight_vector().shape)
zero_vector.shape


In [ ]:
print('Train loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net, trainloader)))
print('Fixed Train loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net, fixed_trainloader)))
print('Test loss mean: {:.6f} std: {:.6f} acc:{:.2f}'.format(*test(net, testloader)))

In [ ]:
b1, b2 = np.random.normal(loc = zero_vector, scale=1),np.random.normal(loc = zero_vector, scale=1)
print(b1.shape)
b1, b2 = net.filterwisely_normalize(b1), net.filterwisely_normalize(b2)
b1.shape

In [ ]:
def plot_2d_landscape(loss_map, std_map,r=1.0, title='Title',):
    scale = loss_map.shape[0]
    x = np.linspace(-r, r, scale)
    y = np.linspace(-r, r,scale)
    X,Y = np.meshgrid(x,y)

    
    fig, axes = plt.subplots(1,2, figsize=(15,10))
    

    cs = axes[0].contour(X,Y,loss_map, levels=np.logspace(np.log(loss_map.min()), np.log(loss_map.max()),15,base=np.e))
    plt.clabel(cs, inline=1, fontsize=10)
    axes[0].set_title('Loss')
    cs = axes[1].contour(X,Y,std_map, levels=np.logspace(np.log(std_map.min()), np.log(std_map.max()),15,base=np.e))
    plt.clabel(cs, inline=1, fontsize=10)
    axes[1].set_title('Std')
    fig.suptitle(title)



In [ ]:
cifar10_sgd_1_11 = eval_plot_2d_landscape(net, Model(in_size=(3,32,32)), fixed_trainloader, (b1,b2), r=1.0, scale=11, title='Title')

In [ ]:
cifar10_sgd_1_101 = eval_plot_2d_landscape(net, Model(in_size=(3,32,32)), fixed_trainloader, (b1,b2), r=1.0, scale=51, title='Title')

In [ ]:
import pickle
pickle.dump((cifar10_sgd_1_11,cifar10_sgd_1_101),open('cifar10_11_51','wb'))